# <div align="center"> RAMP: AIRLINE CARRIERS PERFORMANCE PROJECT </div>
---
Authors: Soumaya SABRY | Alexandre ZAJAC | Maxime LEPEYTRE | Olivier BOIVIN | Yann KERVELLA

<div style="text-align: center">
<img src="images/airplane.png?raw=1" width="800px" />
</div>

# Table of contents
1. [Introduction](#Introduction) 
    - [Storytelling](#Story)
    - [Main Dataset](##Main_Dataset) (peut etre un peu de viz?)
    - [Weather Data](#Weather_Data)
    - [Tweeter Data](#Tweeter_Data)
2. [Data exploration](#Data_exploration)
    - [Import Python libraries](#Import)
    - [Download the data](#Download_data)
    - [Data Base](#DB) 
    - [K-nearest neighbors algorithm](#KNN)
    - [Use of Tweeter](#lars)
    - [Resgression algorithm](#lassolars)
4. [Performance metric](#Metric)
3. [Submission](#Submission) 

# Introduction <a class="anchor" id="Introduction"></a>

## Storytelling <a class="anchor" id="Story"></a>

Recently, big oil companies like BP, Total, announced that they want to reach the goal of **zero carbon** emissions, following the declarations of the governments. 
During this period of Covid-19, the ecological transition has been accelerated as we can see more investment in the renewable energy sector and especially less investment in the oil sector. To achieve this goal, each sector must reduce its emissions.

 We have chosen to focus our study on the airline industry. Following an a report on [IEA](https://www.iea.org/) (International Energy Agency). Since 2000, commercial passenger flight activity has grown by about 5% per year, while CO2 emissions rose by 2% per year, thanks to operational and technical efficiency measures. The energy intensity of commercial passenger aviation has decreased 2.8% per year ib average, but improvements have slackened over time. As we can see on this [figure](https://www.iea.org/data-and-statistics/charts/energy-intensity-of-passenger-aviation-in-the-sustainable-development-scenario-2000-2030) below, where it exposes the Sustainable Development Scenario (one RTK is generated when a metric tonne of revenue load is carried one km). 

<div style="text-align: center">
<img src="images/IEA_2.PNG?raw=1" width="800px" />
</div>

We uses the ICAO carbon calculator to understand which factors can impact the C02 emissions:


$$
C_{O2} \; per \; pax = 3.16 * (total\;fuel*pax\;to\;freight\;factor)*(number\;of\;y-seats*pax\,load\;factor)

$$

We choose to focus on the load factor as 2020 impacted it trought the pandemic
as shown in the figure below from the [report](https://ec.europa.eu/transport/sites/transport/files/legislation/com20200558_allocation_of_slots.pdf) from the commission to the european parleament and the council . We can see the average load factor for a pool of European air carriers droped from 80% in week 9 to 26% in week 15. THis is due not only to air carriers flu less, but the few remain underbooked. We want to focus our study on the load factor and it impacts on the C02 emissions.

We have chosen to focus our model on forecasting the evolution of the load factor. The aim of this forecast is to help airlines adapt their flight plans to optimize their load factor and reduce their emissions. During covid-19, their load factor is low and we want to find a way to boost the load factor (add new path or delete some paths).

<div style="text-align: center">
<img src="images/MeanLF.png?raw=1" width="800px" />
</div>


***What is the Passenger Load Factor ?***

$Load \space{Factor}  = \frac{RPM}{ASM}$

• RPM : Revenue Passenger Miles ($ RPM = Passengers * Distance $)

• ASM : Available Seat Miles ($ ASM = Available\space{Seats} * Distance $)

To calculate the load factor for a single flight, you can get rid of the distance term, however to calculate it over a month for a company, you've got to take into account the distance factor.

This factor is important for all transport system companies, as it assesses the performance. The majority of the revenue is made by selling tickets, to cover the high fixed costs and make profits. 

It is also important to maximize the load factor for environmental reasons, it maximizes the **fuel efficiency**. As passengers's weight represent a very low proportion of the total weight of an airplane, increasing the number of passengers lowers the fuel consumption per passengers.

Investors also look at the Airlines Load Factors as profitability indicators as airlines have very high fixed costs compared to railways companies for example. Having a high load factor is essential for any company's success. 
We can also notice that ASM is a useful metric to measure an airline's capacity to generate revenues.

We call "**break-even Load Factor**" the minimum load factor needed to cover the expenses for one flight.


## Main Dataset <a class="anchor" id="Main_Dataset"></a>  : Bureau of Transportation Statistics (BTS)

We first to chose a baseline place for building a dataset given our problem, and after a lot of investigation, we chose to go for an US-based dataset since they provide an enormous amount of databases, tables and features to chose from. Out logic was first breadth-oriented, since there exist a lot of different websites that source aviation data, but we finally settled on the [Bureau of Transportation Statistics](https://www.transtats.bts.gov/) because it's the source for a lot of others datasets.

Preeminent source of statistics on aviation, multimodal freight activity and transportation economics.
It is a very popular and used source of informations for both political, commercial and public use.
It is part of the US Department of Transportation (DOT).

We chose to use the dataset (**T-100 Domestic Segment**) containing detailed informations on the flights for US Carriers only like  `PASSENGERS, SEATS, PAYLOAD, DESTINATION_CITY, AIRCRAFT_TYPE, ...`
We've transformed this dataset so it was indexed by `DATE` (month) and by `UNIQUE_CARRIER_NAME` on the period 2014-2019. We gathered the informations by month and by companies and enriched the dataset with statistical features, temperatures, top destinations, etc... from there. Our goal was to extract as much pertinent features as possible to make a relevant dataset that would be interesting to work on.

Since part of the data we wanted on the website wasn't directly available for all the given years (2014-2020), we put up a simple script that copied the network request from the transtats website in Node.JS (chrome only let us get the request via curL or Node.JS fetch), and from there we saw that we could modify the fields and filter parameters from the SQL query the site was running, to get all the fields we wanted.


In [ ]:
##pas joli de metrre ca ici
def read_n_clean(name):
  df = pd.read_csv(name)
  # Drop Unnamed Columns
  df.drop(df.columns[df.columns.str.contains('unnamed', case=False)], axis = 1, inplace = True)
  df['DATE'] =  pd.to_datetime(df[['YEAR', 'MONTH']].assign(DAY=1))
  # We pick only data from 2014 to 2019
  date_ref1 = datetime.datetime(2014, 1, 1)
  date_ref2 = datetime.datetime(2020, 1, 1)
  df = df[(df['DATE'] >= date_ref1) & (df['DATE'] < date_ref2)]
  # We pick only flights with available seats > 10
  df = df[df["SEATS"] > 10].reset_index(drop=True)  
  # We pick only flights with non-null distance travelled 
  df = df[df["DISTANCE"] > 0].reset_index(drop=True) 
  # We dropped the Hageland Airline data -> the company has shut down since 2008 and has very low amounts of tweets
  df = df[df['UNIQUE_CARRIER'] != "H6"]
  # Load-Factor Creation
  df["RPM"] = df["PASSENGERS"] * df["DISTANCE"]
  df["ASM"] = df["SEATS"] * df["DISTANCE"]
  return df

In [ ]:
main_df = read_n_clean("data/T_100_Domestic_Segment_All_Years_Extended.csv")
main_df.head()

### **Additional Features : Weather Data with Meteostat**

We made the choice to enrich our base dataset with meteorological features, since we thought they could have a great influence on the load factor for an airline/carrier. 

On the one hand, snow, rain and crosswinds mean that air traffic controllers have to increase the gap between planes that are landing, reducing the number of aircraft that an airport can manage. The same weather can make it slower and more difficult for the planes to taxi between runway and terminal building. On the other hand, the average temperature and sunshine level on the arrival destination can influence the choice of passengers for a given carrier.

A lot of APIs provide daily or weekly live forecast for weather data but few provide an historical record for free. We finally found [the meteostat website](https://dev.meteostat.net/) that provides JSON APIs, a python library and even a web app to directly visualize weather data accross the entire globe, for free (long live open-source)!

Because the weather APIs of meteostats have a nice built-in function to determine which weather station is the closest to the city where we want to query the weather data for, we used the [geopy library](https://geopy.readthedocs.io/en/stable/) to get all the coordinates of the possibles destinations for the flights of our dataset. The weather data is then retrieved from these given weather stations.

We chose the features in line with our problem: `tavg, tmax, tsun, snow, wspd and prcp` (as given [here](https://dev.meteostat.net/python/daily.html#data-structure)). Since our data has a monthly granularity, we directly aggregated the data with the python API meteostats provides, and normalized the data to account for temporal outage of some of the sensors from the weather stations.

In [ ]:
weather_df =  pd.read_csv("data/weather_dataset.csv")
weather_df.head()

### **Additional Features : Tweeter Data with web-scraping**

It's not uncommon to include Twitter data in machine learning challenges. Since we are forecasting a load factor, which is a proxy for the performance of an airline carrier, we thought that incorporating tweets mentionning the chosen airlines could give more breadth to the features of our dataset. More precisely, by performing sentiment analysis on the tweets of a given airline, one could get a sense and new feature to test for the prediction model.

There exist an API for getting tweets from Twitter, so we signed up for a developer account, but the free version only allows to get historical data up to 7 days. So we switched to libraries performing web scraping to get tweets, and there are a lot of them (probably too much), and after a lot of trail and error we settled on [scweet](https://github.com/Altimis/Scweet), forked the repository and made a custom script for our requirements.

Since scraping 6 years of tweets for 23 carriers can be very long, our was parallelized to launch one headless chroime instance per core, with an interval of 30 days of tweets to scrape as we found this to give a good trade-off. We then set-up another script to convert all these tweets Dataframes into dataframes with sentiments probabilities (negative, neutral or positive), and for this we used the amazing [Hugging Face Transformers library](https://huggingface.co/) and more particularly the [Roberta model trained on tweets](https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment) to extract 3 sentiment probabilities for each tweets. As some carriers are lot less present in the Twitter feed, there are some discrepencies of dataset size and we found ways to handle them.

In [ ]:
tweeter_df =  pd.read_csv("data/tweets_dataset.csv")
tweeter_df.head()

# Data exploration <a class="anchor" id="Data_exploration"></a>

In [ ]:
#!pip install -U -r requirements.txt --user

In [ ]:
# a changer si besion
import pandas as pd
import numpy as np
import random
from scipy.stats import pearsonr
import datetime
from tqdm.notebook import tqdm
from sklearn.preprocessing import StandardScaler
from dateutil.relativedelta import relativedelta
#custom
from problem import get_data

## Download the data (optional) <a class="anchor" id="Download_data"></a>

If the data has not yet been downloaded locally, uncomment the following cell and run it. Be patient. This may take a few minutes. The download size is less than 150 MB.

In [ ]:
# !python download_data.py

You should now be able to find the `test` and `train` folders in the `data/` directory

## Data Base <a class="anchor" id="DB"></a>

In [ ]:
X_train, y_train = get_data('train')
X_train.columns ## plus explication

In [ ]:
X_train.shape